In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,354 kB]
Get:12 http://ppa.launchpad.net

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [40]:
import pandas as pd
d = {'x': [7, 17, 27, 37, 47, 57,67,77,87,97,107,117,127,137, 147], 'label': [7, 18, 16, 24, 22, 30, 28, 36, 34, 42, 40, 48, 46, 54, 52]}
df = pd.DataFrame(data=d)
df1=spark.createDataFrame(df)
df1.columns

['x', 'label']

In [41]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

assembleur=VectorAssembler(inputCols=['x'],outputCol='features')
df_nouv=assembleur.transform(df1)
df_nouv.printSchema()

root
 |-- x: long (nullable = true)
 |-- label: long (nullable = true)
 |-- features: vector (nullable = true)



In [42]:
df_nouv.select(['features','label']).show()

+--------+-----+
|features|label|
+--------+-----+
|   [7.0]|    7|
|  [17.0]|   18|
|  [27.0]|   16|
|  [37.0]|   24|
|  [47.0]|   22|
|  [57.0]|   30|
|  [67.0]|   28|
|  [77.0]|   36|
|  [87.0]|   34|
|  [97.0]|   42|
| [107.0]|   40|
| [117.0]|   48|
| [127.0]|   46|
| [137.0]|   54|
| [147.0]|   52|
+--------+-----+



In [43]:
appr, test = df_nouv.randomSplit([0.75, 0.25])
print(f"Taille données d'apprentissage : {appr.count()}" )
print(f"Taille données de test : {test.count()}" )

Taille données d'apprentissage : 8
Taille données de test : 7


In [44]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()
lr_model = lr.fit(appr)

In [45]:
pred_df=lr_model.transform(test)
pred_df.show()

+---+-----+--------+------------------+
|  x|label|features|        prediction|
+---+-----+--------+------------------+
|  7|    7|   [7.0]|13.419689119170979|
| 67|   28|  [67.0]| 31.13101406365655|
| 87|   34|  [87.0]| 37.03478904515174|
| 97|   42|  [97.0]|39.986676535899335|
|107|   40| [107.0]| 42.93856402664693|
|117|   48| [117.0]|45.890451517394524|
|127|   46| [127.0]| 48.84233900814212|
+---+-----+--------+------------------+

